In [ ]:
import csv
import itertools
import re
import pandas as pd


#* ENVS
wspr_csv = "2022-01-08-flt8.csv"
time_slot = 8
re_time_slot = f'^..:.{time_slot}'
parsed_csv = f'parsed_{wspr_csv}'

header = []
parsed_dt = []
alt = []


def alt_calc(power,split_ts):
    if split_ts == None:
        alt_m = int(power)*20 #* Type 3 WSPR message decode x 20 (20m per dBm)
    else:
        alt_m = int(power)*300 #* Type 1 or 2 WSPR message decode x 300 (300m per dBm)
    alt_ft = alt_m * 3.28 #* Convert meters to feet
    return (alt_m, int(alt_ft))


with open(wspr_csv, 'r') as file:
    data = csv.reader(file)
    header = next(data)
    for row in data:
        alt = []
        split_dt = row[1].split(' ')
        split_ts = re.search(re_time_slot, split_dt[1])
        alt_m = alt_calc(row[15], split_ts)
        alt.extend(alt_m)
        parsed_dt.append(list(itertools.chain(row, split_dt, alt)))

add_headers = ['date', 'time', 'altitude_m', 'altitude_ft']
header.extend(add_headers)

with open(parsed_csv, "w",newline="") as f:
    writer = csv.writer(f)
    header[1] = 'datetime'
    # print(header)
    writer.writerow(header)
    for row in parsed_dt:
        writer.writerow(row)

In [86]:
#* Adding in a dataframe to work to add the WSPR 3 packet to the Type 1 and 2

parsed_df = pd.read_csv(parsed_csv, parse_dates=["datetime"])
parsed_df.tail()

,id,datetime,band,rx_sign,rx_lat,rx_lon,rx_loc,tx_sign,tx_lat,tx_lon,...,frequency,power,snr,drift,version,code,date,time,altitude_m,altitude_ft
3929,3783950618,2022-01-07 23:38:00,14,VK1FOC/4,-27.104,152.792,QG62jv,KD9ORR,10.479,166.958,...,14097165,23,-18,0,NaN,1,2022-01-07,23:38:00,6900,22632
3930,3783952631,2022-01-07 23:38:00,14,BV7AU,22.771,120.542,PL02gs,KD9ORR,10.479,166.958,...,14097123,23,-25,0,1.3 Kiwi,1,2022-01-07,23:38:00,6900,22632
3931,3783952646,2022-01-07 23:38:00,14,BV7YA,22.771,120.542,PL02gs,KD9ORR,10.479,166.958,...,14097093,23,-26,0,1.3 Kiwi,1,2022-01-07,23:38:00,6900,22632
3932,3783956963,2022-01-07 23:38:00,14,AI6VN/KH6,20.979,-156.542,BL10rx,KD9ORR,10.063,166.542,...,14097133,23,-23,0,WD_3.0a,1,2022-01-07,23:38:00,6900,22632
3933,3783953642,2022-01-07 23:40:00,14,VK1FOC/4,-27.104,152.792,QG62jv,KD9ORR,10.063,166.542,...,14097165,53,-21,0,NaN,1,2022-01-07,23:40:00,1060,3476


In [112]:
selection = parsed_df[['datetime','date','time', 'tx_lat', 'tx_lon', 'tx_loc', 'altitude_m']]  ## frame
# selection = parsed_df['time']  ## series
# selection.head()
selection.tail()

,datetime,date,time,tx_lat,tx_lon,tx_loc,altitude_m
3929,2022-01-07 23:38:00,2022-01-07,23:38:00,10.479,166.958,RK30,6900
3930,2022-01-07 23:38:00,2022-01-07,23:38:00,10.479,166.958,RK30,6900
3931,2022-01-07 23:38:00,2022-01-07,23:38:00,10.479,166.958,RK30,6900
3932,2022-01-07 23:38:00,2022-01-07,23:38:00,10.063,166.542,RK30gb,6900
3933,2022-01-07 23:40:00,2022-01-07,23:40:00,10.063,166.542,RK30gb,1060


In [113]:
selection.dtypes

datetime      datetime64[ns]
date                  object
time                  object
tx_lat               float64
tx_lon               float64
tx_loc                object
altitude_m             int64
dtype: object

In [114]:
type(selection)

pandas.core.frame.DataFrame

In [138]:
# for a frame

eights = selection['time'].str.extract(r'(^..:.8.*)', expand = False)
zeros = selection['time'].str.extract(r'(^..:.0.*)', expand = False)
# eights
# zeros


In [ ]:
# Works for a series
selection.tail()
eights = selection.str.extract(r'(^..:.8.*)')
eights


In [141]:
frames = [eights, zeros]
combined = pd.concat(frames)
combined.tail()

3929         NaN
3930         NaN
3931         NaN
3932         NaN
3933    23:40:00
Name: time, dtype: object

Now want to take the altitude in the :08 mins and add to the following :00 mins. to get teh precise altitude.

```
date,time,altitude_m,altitude_ft
2022-01-07,23:38:00,6900,22632
2022-01-07,23:40:00,1060,3476
```

For example the precise altitude is `7960 m`